## Lecture Notes

We want to estimate the random variable X from a noisy oberserved variable Y

$$
\hat{\mathbf{x}}=A\mathbf{y}+\mathbf{b}
$$

Such that the error \\(\mathbb{E}[||\mathbf{x}-\mathbf{\hat{x}}||]\\) is minimized

By defining the zero mean vectors \\(\mathbf{\tilde{x}}=x-\mu_x\\) and \\(\mathbf{\tilde{y}}=y-\mu_y \\) we rewrite

$$
\hat{\mathbf{x}}=A(\mathbf{\tilde{y}}+\mu_y)+\mathbf{b}=A\mathbf{\tilde{y}}+(A\mu_y+\mathbf{b})
$$

To preserve the mean we set \\(A\mu_y+\mathbf{b}=\mu_x\\)

$$
\hat{\mathbf{x}}=A\mathbf{\tilde{y}}+\mu_x
$$

And we find the optimal \\(\mathbf{b}=\mu_x-A\mu_y\\)

Then we can work only on the \\(A\\) term when optimizing.

$$
\mathbb{E}[||\mathbf{\tilde{x}} - A\mathbf{\tilde{y}}||]=\mathbf{E}[(\mathbf{\tilde{x}} - A\mathbf{\tilde{y}})^T(\mathbf{\tilde{x}} - A\mathbf{\tilde{y}})]
$$

This we can differentiate with with respect to \\(A\\) and set it to zero which gives us

$$
A=C_{yy}^{-1}C_{xy}
$$

Where \\(C_{xy}\\) and \\(C_{yy}\\) are the cross and auto correlation matrices respectively.

This works for any joint distribution of $X$ and $Y$ and only requires the first two moments of the distributions, mean and covariance.

$$
\mathbf{\tilde{x}}=\mu_x+C_{yy}^{-1}C_{xy}(y-\mu_y)
$$

### Special case for additive Gaussian noise(Wiener filter)

If we assume the model \\(\mathbb{y}=H\mathbb{x}+n\\) where $n$ is zero mean noise independent of x with covariance $C_{nn}$

Then by linear transformation we can find the cross covariance defined by the linear transformation 
$$
C_{xy}=C_{xx}H^T
$$

And the autocovariance(See lecture slides for proofs)

$$
C_{yy}=HC_{xx}H^T+C_{nn}
$$

Since we perform a linear transformation

We can plug this into the original solution

$$
\mathbf{\tilde{x}}=\mu_x+C_{xx}H^T(HC_{xx}H^T+C_{nn})^{-1}(y-\mu_y)
$$

### MSE For linear Minimum Mean Square Estimators

The MSE is defined as 

$$
\mathbb{E}[(x-\tilde{x})^2]
$$

After minimizing the MSE and solving for the estimation parameters, the theoretical minimum MSE is obtained by

$$
MSE(\tilde{x})=C_{xx}-C_{xy}C_{xx}^{-1}C_{xy}
$$

For the scalar case, this reduces to

$$
MSE(\tilde{x})=Var(x)-\frac{Cov(x,y)^2}{Var(y)}
$$

## Exercises

![image.png](attachment:2fc970f2-8591-4378-bc36-e7896f02547f.png)

In [0]:
def ar_one(w, theta):

    N = len(w)

    x = np.zeros_like(w)
    
    x[0] = w[0]
    for i in range(1, N):
        x[i] = theta*x[i-1]+w[i]

    return x

In [0]:
# a and b)
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

N = 1000
nlags = 20
sigma_w = 1

w_ = []
x_ = []

r_ = [0.1, 0.5, 0.9]
for r in r_:
    w = np.random.normal(0, 1, N)

    x = ar_one(w, r)

    w_ += [w]
    x_ += [x]
    
    acorr = acf(x, nlags=nlags)
    r_0 = sigma_w**2/(1-r**2)
    acorr_theoretical = [r_0*r**np.abs(k) for k in range(nlags+1)]
    acorr_theoretical = [a/acorr_theoretical[0] for a in acorr_theoretical]

    
    plt.figure()
    plt.plot(w, label="Noise")
    plt.plot(x, label="AR(1)")
    plt.title(f"AR(1) = {r}")
    plt.legend()

    plt.figure()
    plt.plot(acorr, label="Empirical")
    plt.plot(acorr_theoretical, label="Theoretical")
    plt.title(f"Autocorrelation r={r}")
    plt.legend()

We want to estimate some signal $x$ from several observations $\mathbf{y_n}$

$$
\hat{x_n}=\mathbf{h}^T\mathbf{y_n}
$$

From LMMSE we know that 

$$
\hat{x}_n=C_{xy}C_{yy}^{-1}\mathbf{y}
$$

Where the covariance matrices becomes correlation matrices

$$
\mathbf{h}=R_{yy}^{-1}\mathbf{r}_{xy}
$$

Because we estimate a scalar from a vector, we take the first row of the cross covariance matrix instead of the whole matrix.

We can estimate both present(denoising), future(prediction) and previous(smoothing) values using the Wiener filter. This comes down to how the cross correlation vector is defined

$\tau=0$ is denoising, $\tau>0$ is prediction and $\tau < 0$ is smoothing

### Filtering

For filtering, we use the following model

$$
y_n=x_n+w_n
$$

Where $x_n$ is the signal and $w_n$ is noise uncorrelated to the signal.

We know that $C_{xx}=R_{ss}+R_{ww}$ since signal and noise is uncorrelated.
The observations are formed as the vector $\mathbf{y}_n=[y[0],..,y[n]]$

We then define the covariance matrix 

$$
C_{xy}=\mathbb{E}[x_n\mathbf{y}_n]=\mathbb{E}[x_n(\mathbf{x}_n+\mathbf{w}_n)]=\mathbb{E}[x_n\mathbf{x}_n]+\mathbb{E}[x_n\mathbf{w}_n]=\mathbb{E}[x_n\mathbf{x}_n]=[r_{xx}(n),..,r_{xx}(0)]=\mathbf{r}_{xx}' 
$$

The small "'" sign means that the vector is flipped. That is fine by us, since Python flips the second array when convolving.

Since $w_n$ and $x_n$ are uncorrelated $\mathbb{E}[s_n\mathbf{x}_n]$=0

This gives us the Wiener-Hopf equations

\begin{equation}
\underbrace{\begin{bmatrix}
r_{yy}[0]   & r_{yy}[-1] & \cdots & r_{yy}[1-N]\\[4pt]
r_{yy}[1]   & r_{yy}[0]  & \cdots & r_{yy}[2-N]\\[4pt]
\vdots      & \vdots     & \ddots & \vdots      \\[4pt]
r_{yy}[N-1] & r_{yy}[N-2]& \cdots & r_{yy}[0]
\end{bmatrix}}_{\mathbf r_{yy}}
\;
\underbrace{\begin{bmatrix} h_0\\ h_1\\ \vdots\\ h_{N-1}\end{bmatrix}}_{\mathbf h}
\;=\;
\underbrace{\begin{bmatrix}
r_{xx}[0]\\[4pt]
r_{xx}[1]\\[4pt]
\vdots\\[4pt]
r_{xx}[N-1]
\end{bmatrix}}_{\mathbf r_{xx}}.
\end{equation}

We can calculate $r_{xx}$ from the estimated $r_{yy}$. From the signal model $y_n=x_n+w_n$ we calculate the covariance to be $Cov_y(k)=Cov_x(k)$ for $k \neq 0$ and $Cov_y(0)=Cov_x(0)+\sigma_w^2$ for $k=0$. Hence $r_{xx}(k)=r_{yy}(k)$ for $k \neq 0$ and $r_{xx}(0)=r_{yy}(0)-\sigma_w^2$

### Prediction

We want to estimate $x=y_{N-1+I}$ from the last $[y[0],..,y[N-1]]$ observations

The covariance matrix becomes

$$
C_{xy}=\mathbb{E}[x_n \mathbf{y}_n]=\mathbb{E}[y_{N-1+I} \mathbf{y}_n]=\mathbb{E}[y_{N-1+I} [y_0,..y_{N-1}]]=[r_{yy}(N-1+I),..r_{yy}(I)]=\mathbf{r}_{yy}'
$$

Now the Wiener-Hopf equations are 

\begin{equation}
\underbrace{\begin{bmatrix}
r_{yy}[0]   & r_{yy}[-1] & \cdots & r_{yy}[1-N]\\[4pt]
r_{yy}[1]   & r_{yy}[0]  & \cdots & r_{yy}[2-N]\\[4pt]
\vdots      & \vdots     & \ddots & \vdots      \\[4pt]
r_{yy}[N-1] & r_{yy}[N-2]& \cdots & r_{yy}[0]
\end{bmatrix}}_{\mathbf r_{yy}}
\;
\underbrace{\begin{bmatrix} h_0\\ h_1\\ \vdots\\ h_{N-1}\end{bmatrix}}_{\mathbf h}
\;=\;
\underbrace{\begin{bmatrix}
r_{yy}[I]\\[4pt]
r_{yy}[I+1]\\[4pt]
\vdots\\[4pt]
r_{yy}[I+N-1]
\end{bmatrix}}_{\text{RHS prediction vector}}.
\end{equation}

In [0]:
# c, d, e and f)
from statsmodels.tsa.stattools import acf

P = 3
tau = 1

r_ = [0.1, 0.5, 0.9]
for r in r_:
    w = np.random.normal(0, 1, N)

    x = ar_one(w, r)

    acf_values = acf(x, nlags=P+tau)
    
    cov = np.zeros((P,P))
    cross = np.zeros(P)
    for i in range(P):
        cross[i] = acf_values[i+tau]
        for j in range(P):
            cov[i, j] = acf_values[np.abs(i-j)]

    h = np.linalg.inv(cov) @ cross

    x_hat = np.convolve(x, h, mode='full')[:-(P-1)]

    e = x-x_hat
    mse = np.mean(e**2)

    plot_num = 100
    
    plt.figure()
    plt.plot(x[:plot_num], label="x")
    plt.plot(x_hat[:plot_num], label="x one-step-predicted")
    #plt.plot(e, label="error")
    plt.legend()
    plt.title(f"r={r}, MSE={mse:.3f}")

    

As r grows larger, a better prediction performance is expected, since the process is less random.

![image.png](attachment:65a5eba3-a5b9-4847-afc9-376946d18c36.png)
![image.png](attachment:125da7ae-ae1e-40a8-9d02-f3b2ca9b849a.png)

In [0]:
# 1)
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

N = 1000
nlags = 20
sigma_w = 1
sigma_e = 1
w_ = []
x_ = []

r_ = [0.1, 0.5, 0.9]
for r in r_:
    w = np.random.normal(0, 1, N)
    e = np.random.normal(0, 1, N)
    x = ar_one(w, r)

    z = x + e

    w_ += [w]
    x_ += [x]
    
    acorr = acf(z, nlags=nlags)

    r_0 = sigma_w**2/(1-r**2)
    acorr_theoretical = [(r_0+sigma_e**2) if k==0 else r_0*r**np.abs(k) for k in range(nlags+1)]
    acorr_theoretical = [a/acorr_theoretical[0] for a in acorr_theoretical]

    
    plt.figure()
    plt.plot(w, label="Noise")
    plt.plot(x, label="AR(1)")
    plt.title(f"AR(1) = {r}")
    plt.legend()

    plt.figure()
    plt.plot(acorr, label="Empirical")
    plt.plot(acorr_theoretical, label="Theoretical")
    plt.title(f"Autocorrelation r={r}")
    plt.legend()

In [0]:
# 2, 3, 4 and 5)
from statsmodels.tsa.stattools import acf

N = 1000
P = 3
tau = 0
sigma_w = 1
r_ = [0.1, 0.5, 0.9]
var_e_ = [0.25, 1.0, 9.0]
for var_e in var_e_:    
    for r in r_:
    
        w = np.random.normal(0, 1, N)
        e = np.random.normal(0, np.sqrt(var_e), N)
        z = ar_one(w, r)
        x = z + e
        
        r_0 = sigma_w**2/(1-r**2)

        acorr = acf(x, nlags=20, fft=True) * np.var(x)
        
        plt.figure()
        plt.plot(acorr)
        
        # We compute the cross an correlations analytically
        rxx = np.zeros((P,P))
        rzz = np.zeros(P)
        for i in range(P):
            
            # For the cross correlation, the noise term cancels out
            rzz[i] = acorr[i+tau]
            if i+tau == 0:
                rzz[i+tau] -= var_e
            
            for j in range(P):
                rxx[i, j] = acorr[np.abs(i-j)]
    
        h = np.linalg.solve(rxx, rzz)
        z_hat = np.convolve(x, h, mode='full')[:-(P-1)]
        
        e = z-z_hat
        mse = np.mean(e**2)

        # Using scalar version
        mse_theoretical_scalar = r_0 - (r_0**2)/(r_0 + var_e)

        # Using vector version
        mse_theoretical = rzz[0] - rzz.T @ np.linalg.inv(rxx) @ rzz

        #print(rxx[0][0], np.var(x))
        
        plot_num = 100

        title = f"r={r:.2f}, var_e={var_e:.2f}, MSE={mse:.3f}, MSE Expected={mse_theoretical:.3f}"
        print(title)
        
        plt.figure()
        plt.plot(z[:plot_num], label="z")
        plt.plot(x[:plot_num], label="x", alpha=0.3)
        plt.plot(z_hat[:plot_num], label="z denoised(From x)")
        plt.legend()
        plt.title(title)

        '''
        plt.figure()
        plt.plot(z-x_hat, label="Error")
        plt.title("Error)
        '''